# Lab 1

In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
# data from csv
auto_df = pd.read_csv('ST83446_Auto-mpg_dataset.csv')

In [6]:
auto_df.shape

(54, 9)

In [7]:
#peak inside
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,Car model
0,15.0,8,383,170,3563,10.0,70,1,dodge
1,15.0,8,400,150,3761,9.5,70,1,chevrolet
2,14.0,8,455,225,3086,10.0,70,1,buick
3,21.0,6,200,85,2587,16.0,70,1,ford
4,27.0,4,97,88,2130,14.5,70,3,datsun
